#### Key Summary  

1. Temperatures appear to reach the highest near 20 degrees latitude.
2. Temperatures are higher at latitudes closer to the equator. Temperatures are lower towards the earth's poles.
3. The datasets for cloudiness and wind speed don't seem to have any correlation to latitude, or proximity to the equator.

In [ ]:
# Dependencies
import matplotlib.pyplot as plt
import requests
from scipy import stats
import pandas as pd
from config import api_key
import numpy as np
from ratelimiter import RateLimiter
from citipy import citipy

output_data_file = "output_data/cities.csv" #Output csv file 

# Range of latitudes and longitudes
lat_range = (-70, 70) #using -70/80 as there are less cities near the poles
lng_range = (-180, 180)


In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1000)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1000)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

In [ ]:
#Perform API Calls------------------------------------------- 

#Lists and counters
city_list = []
cloud_list = []
country_list = []
date_list = []
humidity_list = []
lats_list = []
lngs_list = []
temp_max_list = []
wind_speed_list = []
index_counter = 0
set_counter = 1  
rate_limiter = RateLimiter(max_calls=5, period=60)


#Perform API Calls-------------------------------------------  


base_url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"
query_url = f"{base_url}appid={api_key}&units={units}&q="


#For loop matching city names with city_list using enumerate method

for index, city in enumerate(cities, start = 1):
    with rate_limiter:
        try:
            response = requests.get(query_url + city).json()
            city_list.append(response["name"])
            cloud_list.append(response["clouds"]["all"])
            country_list.append(response["sys"]["country"])
            date_list.append(response["dt"])
            humidity_list.append(response["main"]["humidity"])
            lats_list.append(response["coord"]["lat"])
            lngs_list.append(response["coord"]["lon"])
            temp_max_list.append(response['main']['temp_max'])
            wind_speed_list.append(response["wind"]["speed"])

            if index_counter > 49:
                index_counter = 0
                set_counter = set_counter + 1

            else:
                index_counter = index_counter + 1

            print(f"Processing Record {index_counter} of Set {set_counter} : {city}")

        except(KeyError, IndexError):
            print("City not found. Skipping...")
            
            

In [ ]:
#Create a dataframe using information from data retrieval
weather_data = pd.DataFrame({
    "City" : city_list,
    "Lat" : lats_list,
    "Lng" : lngs_list,
    "Max Temp" : temp_max_list,
    "Humidity" : humidity_list,
    "Clouds" : cloud_list,
    "Wind Speed" : wind_speed_list,
    "Country" : country_list,
    "Date" : date_list    
})

#Count weather data
weather_data.count()

In [ ]:
weather_data.head()

In [ ]:
#Save weather data to a cities csv file
weather_data.to_csv("output_data/cities.csv", index=False)

The first requirement is to create a series of scatter plots to showcase the following relationships:

Temperature (F) vs. Latitude
Humidity (%) vs. Latitude
Cloudiness (%) vs. Latitude
Wind Speed (mph) vs. Latitude

In [ ]:
#Temperature (F) vs. Latitude scatter plot
plt.scatter(weather_data["Lat"], weather_data["Max Temp"], color = "steelblue", edgecolor = "black")

plt.title("Temperature (F) vs. Latitude")
plt.xlabel("Latitude")
plt.ylabel("Max Temperature (F)")
plt.grid(True)
plt.savefig("../Images/City Latitude vs Max Temperature.png")

In [ ]:
#Wind Speed (mph) vs. Latitude scatter plot
plt.scatter(weather_data["Lat"], weather_data["Wind Speed"], color = "steelblue", edgecolor = "black")

plt.title("Wind Speed (mph) vs. Latitude")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (%))")
plt.grid(True)
plt.savefig("../Images/City Latitude vs Wind Speed.png")

In [ ]:
#Cloudiness (%) vs. Latitude scatter plot
plt.scatter(weather_data["Lat"], weather_data["Clouds"], color = "steelblue", edgecolor = "black")

plt.title("Cloudiness (%) vs. Latitude")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%))")
plt.grid(True)
plt.savefig("../Images/City Latitude vs Cloudiness.png")

In [ ]:
#Humidity (%) vs. Latitude scatter plot
plt.scatter(weather_data["Lat"], weather_data["Humidity"], color = "steelblue", edgecolor = "black")

plt.title("Humidity (%) vs. Latitude")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%))")
plt.grid(True)
plt.savefig("../Images/City Latitude vs Humidity.png")

The second requirement is to run linear regression on each relationship. This time, separate the plots into Northern Hemisphere (greater than or equal to 0 degrees latitude) and Southern Hemisphere (less than 0 degrees latitude):

Northern Hemisphere - Temperature (F) vs. Latitude
Southern Hemisphere - Temperature (F) vs. Latitude
Northern Hemisphere - Humidity (%) vs. Latitude
Southern Hemisphere - Humidity (%) vs. Latitude
Northern Hemisphere - Cloudiness (%) vs. Latitude
Southern Hemisphere - Cloudiness (%) vs. Latitude
Northern Hemisphere - Wind Speed (mph) vs. Latitude
Southern Hemisphere - Wind Speed (mph) vs. Latitude